<a href="https://colab.research.google.com/github/cuong3004/Trash_Cuong/blob/main/NMS_yoloV5_tflite.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [ ]:
import tensorflow as tf

In [ ]:
class NMS(tf.Module):
    @tf.function(input_signature=[tf.TensorSpec(shape=[1, 2268, 7], dtype=tf.float32)])
    def __call__(self, out):
        # print(out.shape)
        # xyxy, classes, scores = self.YOLOdetect(out)
        # print(classes.dtype)
        # print(scores.dtype)
        # return xyxy, classes, scores

        xyxy, classes, scores = self.YOLOdetect(out)

        boxes = tf.stack(xyxy, -1)

        # print(classes)
        selected_indices = tf.image.non_max_suppression(
            boxes, scores, 50, 0.5)
        
        # ptin
        
        selected_boxes = tf.gather(boxes, selected_indices)
        selected_classes = tf.gather(classes, selected_indices)
        selected_scores = tf.gather(scores, selected_indices)
        

        print(selected_boxes.dtype, selected_classes.dtype, selected_scores.dtype)
        print(selected_boxes.shape)

        return selected_boxes, selected_classes, selected_scores
        
    
    def classFilter(self, classdata):

        # print(classdata.shape)
        # classes = []  # create a list
        # for i in range(classdata.shape[0]):         # loop through all predictions
        #     classes.append(classdata[i].argmax())   # get the best classification location
        # # return classes  # return classes (int)
        # print(classdata)
        # print(tf.math.argmax(classdata, -1))
        # return tf.math.argmax(classdata, -1)

        classes = tf.argmax(classdata, -1)
        classes = tf.cast(classes, tf.float32)
        return classes
        
    
    
    def YOLOdetect(self, output_data):  # input = interpreter, output is boxes(xyxy), classes, scores
        output_data = output_data[0]                # x(1, 25200, 7) to x(25200, 7)
        # print(output_data[..., :4].shape)
        boxes = tf.squeeze(output_data[..., :4])  # boxes  [25200, 4]
        # print(boxes.shape)
        scores = tf.squeeze(output_data[..., 4:5]) # confidences  [25200, 1]
        # print(output_data[..., 5:])
        classes = self.classFilter(output_data[..., 5:]) # get classes
        # Convert nx4 boxes from [x, y, w, h] to [x1, y1, x2, y2] where xy1=top-left, xy2=bottom-right
        x, y, w, h = boxes[..., 0], boxes[..., 1], boxes[..., 2], boxes[..., 3] #xywh
        xyxy = [x - w / 2, y - h / 2, x + w / 2, y + h / 2]  # xywh to xyxy   [4, 25200]

         # check data type float32

        return xyxy, classes, scores  # output is boxes(x,y,x,y), classes(int), scores(float) [pr


In [ ]:
nms = NMS()
concrete_func = nms.__call__.get_concrete_function()

<dtype: 'float32'> <dtype: 'float32'> <dtype: 'float32'>
(None, 4)


In [ ]:
converter = tf.lite.TFLiteConverter.from_concrete_functions([concrete_func],
                                                            nms)
tflite_model = converter.convert()

# Save the model.
with open('nms.tflite', 'wb') as f:
  f.write(tflite_model)

<dtype: 'float32'> <dtype: 'float32'> <dtype: 'float32'>
(None, 4)
INFO:tensorflow:Assets written to: /tmp/tmp7h58k617/assets


INFO:tensorflow:Assets written to: /tmp/tmp7h58k617/assets
